In [84]:
#Iván andrés Trujillo Abella 
#ivantrujillo1229@gmail.com
#Consolidate Thesis

In [85]:
## Own functions
def error_in_date(x):
    try:
        datetime.strptime(x, "%Y-%m-%d")
        return False
    except:
        return True

def cleantime(x):
    if type(x)==str:
        return x.replace("00:00:00", "").replace(" ", "")
    else:
        return x

def top_cat(k, cat, df):
    otro = []
    cat_relative_dict =  df[cat].value_counts(normalize=True).to_dict()
    for cat in cat_relative_dict:
        if cat_relative_dict[cat]<=k:
            otro.append(cat)
    return otro


def bankyear(df_):
  df = df_.copy()
  years = [2016, 2017, 2018, 2019]
  df['bankrupt-year'] = 0
  for firm in df.index:
    unchanged = True
    for year in years:
      if df.loc[firm, 'evento'+str(year)]==1 and unchanged==True:
        df.loc[firm,'bankrupt-year']= year
        unchanged = False
  return df['bankrupt-year']



def new_firms(current_df, next_df, key):
  firm_first =  set(current_df[key].unique()) -  set(next_df[key].unique())
  firm_second =  set(next_df[key].unique()) -  set(current_df[key].unique())
  return pd.concat([current_df[current_df[key].isin(firm_first)] , next_df[next_df[key].isin(firm_second)]])


# A function
def varInEvent(time_base, prefix, var_time_event, df_, lag=1):
  df = df_.copy()
  df['var*'] = np.nan
  for firm in df.index:
    time_ocurrence = df.loc[firm, var_time_event]
    if time_ocurrence - lag >= time_base:
      df.loc[firm, 'var*'] = df.loc[firm, prefix + str(time_ocurrence-lag)]
  return df['var*']


In [86]:
cd C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets

C:\Users\iván\Desktop\Thesis\Thesis-bankrupt\Datasets


In [87]:
import pandas as pd
import numpy as np

In [88]:
# Loading datasets
dfs = {}
years = [2016,2017,2018,2019]
for year in years:
  dfs['df'+str(year)]  = pd.read_excel('Cáratula-Pymes-'+str(year)+'.xlsx')

In [89]:
for year in years:
  print('#'*10)
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Estado actual']!='EN ETAPA PREOPERATIVA']
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)]['Fecha'] = pd.to_datetime( dfs['df'+str(year)][ 'Fecha de Corte'], format= "%Y-%m-%d")
  dfs['df'+str(year)]  = dfs['df'+str(year)][dfs['df'+str(year)]['Fecha'].dt.month==12] # there is duplicates given present the information two or more times..
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)] =  dfs['df'+str(year)][-pd.isnull(dfs['df'+str(year)]['NIT'])]
  print(dfs['df'+str(year)].shape , year)
  dfs['df'+str(year)]['evento'+str(year)]  = np.where(dfs['df'+str(year)]['Estado actual']!='ACTIVA',1 ,0)
  dfs['df'+str(year)] = dfs['df'+str(year)][['evento'+str(year), 'NIT']]
  print(dfs['df'+str(year)]['NIT'].duplicated().sum())
  print(dfs['df'+str(year)].shape , year)

##########
(17799, 31) 2016
(17723, 31) 2016
(17701, 32) 2016
(17701, 32) 2016
0
(17701, 2) 2016
##########
(14696, 32) 2017
(14656, 32) 2017
(14639, 33) 2017
(14638, 33) 2017
0
(14638, 2) 2017
##########
(14275, 34) 2018
(14229, 34) 2018
(14212, 35) 2018
(14210, 35) 2018
1
(14210, 2) 2018
##########
(19185, 36) 2019
(19140, 36) 2019
(19128, 37) 2019
(19125, 37) 2019
0
(19125, 2) 2019


In [90]:
dfs['df2019']['evento2019'].value_counts() # Here there are 876 firms,  but since declarate the process.

evento2019
0    18249
1      876
Name: count, dtype: int64

In [91]:
for year in range(2016,2020):
    print(dfs['df'+str(year)]['evento'+str(year)].value_counts())

evento2016
0    17193
1      508
Name: count, dtype: int64
evento2017
0    14019
1      619
Name: count, dtype: int64
evento2018
0    13445
1      765
Name: count, dtype: int64
evento2019
0    18249
1      876
Name: count, dtype: int64


In [92]:
full = dfs['df2016'].copy()
for year in range(2017,2020):
    clients= new_firms(full,dfs['df'+str(year)], 'NIT')
    full = full.merge(dfs['df'+str(year)], on='NIT', how='inner', suffixes=(None, 'to_remove'))
    duplicated = [x for x in full.columns if 'to_remove' in x]
    full.drop(columns=duplicated, inplace=True)
    full = pd.concat([full, clients])

In [93]:
full['NIT'].duplicated().sum()

1

In [94]:
full['evento2019'].value_counts()

evento2019
0.0    18250
1.0      876
Name: count, dtype: int64

In [97]:
full

,evento2016,NIT,evento2017,evento2018,evento2019
0,0.0,800000313.0,0.0,0.0,0.0
1,0.0,800000457.0,0.0,0.0,0.0
2,0.0,800001020.0,0.0,0.0,0.0
3,0.0,800001047.0,0.0,0.0,0.0
4,0.0,800001075.0,0.0,1.0,1.0
...,...,...,...,...,...
19180,NaN,901330952.0,NaN,NaN,0.0
19181,NaN,901333114.0,NaN,NaN,0.0
19182,NaN,901341119.0,NaN,NaN,0.0
19183,NaN,901341297.0,NaN,NaN,0.0


In [95]:






"""
for i in range(2,4):
  clients= new_firms(full,dfs['df'+str(i)], 'NIT')
  full = full.merge(dfs['month'+str(i)], on='cedula', how='inner', suffixes=(None, 'to_remove'))
  duplicated = [x for x in full.columns if 'to_remove' in x]
  full.drop(columns=duplicated, inplace=True)
  full = pd.concat([full, clients])
"""

"\nfor i in range(2,4):\n  clients= new_firms(full,dfs['df'+str(i)], 'NIT')\n  full = full.merge(dfs['month'+str(i)], on='cedula', how='inner', suffixes=(None, 'to_remove'))\n  duplicated = [x for x in full.columns if 'to_remove' in x]\n  full.drop(columns=duplicated, inplace=True)\n  full = pd.concat([full, clients])\n"